ArcFace for proteins! <br>
https://arxiv.org/abs/1801.07698

In [3]:
import torch
import numpy as np
import pandas as pd
import os
import h5py
from exabiome.nn.loader import read_dataset, LazySeqDataset
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from model import *
from data import *
from fastai.text.all import *

In [4]:
df_x = pd.read_csv('data/toy_x.csv')
df_y = pd.read_csv('data/toy_y.csv')
df_x.shape, df_y.shape

((19010, 4097), (19010, 2))

In [5]:
class FastDs(Dataset):
    def __init__(self, df_x, df_y):
        self.x = df_x
        self.y = df_y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        x = self.x.iloc[idx][1:]
        y = self.y.iloc[idx][1:]
        return torch.Tensor(x).unsqueeze(0), torch.Tensor(y).to(torch.long).squeeze()

In [6]:
train_ds = FastDs(df_x[:-2000], df_y[:-2000])
valid_ds = FastDs(df_x[-2000:], df_y[-2000:])
len(train_ds), len(valid_ds)

(17010, 2000)

In [7]:
train_ds[0][1].shape

torch.Size([])

In [8]:
train_ds[0][1].squeeze().shape

torch.Size([])

In [9]:
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_ds, batch_size=16, shuffle=False, drop_last=True)
len(train_dl), len(valid_dl)

(1063, 125)

In [10]:
#DataLoade

In [11]:
batch = next(iter(train_dl))
len(batch), batch[0].shape, batch[1].shape

(2, torch.Size([16, 1, 4096]), torch.Size([16]))

In [12]:
model = EffNet_b0(avg_out=120, custom_head=False)
out = model(batch[0])
out.shape

torch.Size([16, 1, 120])

In [13]:
model = EffNet_b0(out_feats=16, )#custom_head=False)
out = model(batch[0])
out.shape

torch.Size([16, 1, 16])

In [29]:
#model

In [14]:
dls = DataLoaders(train_dl, valid_dl)

In [15]:
#learn = Learner(dls, model, loss_func=nn.CrossEntropyLoss())

In [16]:
class compress_cb(Callback):
    def after_pred(self):
        self.learn.pred = self.pred.squeeze(1)

In [17]:
dls.to('cuda');
model.to('cuda');

In [48]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features)).to('cuda')
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        #sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        sine = torch.sqrt(1.0 - torch.pow(cosine,2)).to(cosine.dtype)
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        #one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [15]:
#F.normalize(out.squeeze(2)).shape

torch.Size([16, 18, 16])

In [34]:
F.normalize(out.squeeze(1)).shape

torch.Size([16, 18])

In [41]:
F.normalize(batch[1].unsqueeze(1).to(torch.float)).shape

torch.Size([16, 1])

In [44]:
F.normalize(batch[1].to(torch.float).unsqueeze(1)).shape

torch.Size([16, 1])

In [17]:
#F.linear(out.squeeze(2), batch[1].squeeze(1).to(torch.float)).shape

torch.Size([16, 18])

In [46]:
F.linear(out.squeeze(1), batch[1].unsqueeze(0).to(torch.float)).shape

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x18 and 16x1)

In [18]:
#out.squeeze(2).shape, batch[1].squeeze(1).shape

(torch.Size([16, 18, 16]), torch.Size([16]))

In [19]:
#F.linear(F.normalize(out.squeeze(2)), batch[1].squeeze(1).to(torch.float)).shape

torch.Size([16, 18])

In [49]:
ArcMarginProduct(18, 18)(out.squeeze(1).to('cuda'), batch[1].to(torch.float).to('cuda')).shape

torch.Size([16, 18])

In [21]:
batch[1].shape

torch.Size([16])

In [50]:
class compress_cb(Callback):
    def after_pred(self):
        self.learn.pred = self.pred.squeeze(1)
        print(self.learn.yb[0].shape)
        self.learn.yb = tuple(self.yb[0].squeeze(1))
        print(self.learn.yb[0].shape)

In [29]:
out = learn.model(batch[0])
out.shape

torch.Size([16, 1, 120])

In [26]:
model = EffNet_b0(out_feats=18,)# custom_head=False)
out = model(batch[0])
out.shape, batch[1].shape

(torch.Size([16, 1, 18]), torch.Size([16]))

In [24]:
nn.CrossEntropyLoss()(out.squeeze(1), batch[1])

tensor(3.1197, grad_fn=<NllLossBackward>)

In [29]:
learn = Learner(dls, model, loss_func=nn.CrossEntropyLoss(),
               cbs=[compress_cb], 
                metrics=[accuracy]).to_fp16()

In [16]:
#for x in range(len(valid_dl)):
#    batch = next(iter(valid_dl))
#    out = learn.model(batch[0])
#    print(nn.CrossEntropyLoss()(out, batch[1]))

In [ ]:
#out.dtype

In [1]:
#learn.lr_find()

In [30]:
learn.fine_tune(5)

epoch,train_loss,valid_loss,accuracy,time
0,2.126462,1.835746,0.485000,02:06


epoch,train_loss,valid_loss,accuracy,time
0,2.013575,1.403061,0.588500,02:05
1,2.001874,1.427351,0.664000,02:06
2,1.925462,1.277372,0.695000,02:04
3,1.827261,1.251863,0.767500,01:59
4,1.879972,1.295908,0.726000,02:02


Looks like things are working with a normal classification setup via cross entropy -- let's now see how things work if we use the arcface!

In [64]:
class amp_cb(Callback):
    def __init__(self):
        self.amp = ArcMarginProduct(18, 18)
    def after_pred(self):
        self.learn.pred = self.pred.squeeze(1)
        self.learn.pred = self.amp(self.pred, self.y)

In [62]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(),
               cbs=[amp_cb], 
                metrics=[accuracy]).to_fp16()

In [63]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,nan,nan,0.000000,01:35


In [ ]:
ArcMarginProduct(18, 18)(out.squeeze(1).to('cuda'), batch[1].to(torch.float).to('cuda')).shape